In [ ]:
!pip install laspy scipy matplotlib pykrige rasterio

In [ ]:
#!pip install --force-reinstall laspy[laszip]

In [ ]:
import laspy
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# ---- IDW Interpolation ----
from scipy.spatial import cKDTree


def idw_interpolation(x, y, z, X, Y, power: int = 2, k: int = 10):
    """
    Performs Inverse Distance Weighting (IDW) interpolation using a KDTree for efficiency.

    Pros: Simple, fast, effective for smooth surfaces.
    Cons: May not handle sharp terrain features well.

    Parameters
    ----------
    x, y, z : np.ndarray
        Known data points (coordinates and elevation).
    X, Y : np.ndarray
        Grid points where interpolation is applied.
    power : int, optional
        Power parameter for IDW weighting (default is 2).
    k : int, optional
        Number of nearest neighbors to consider (default is 10).

    Returns
    -------
    np.ndarray
        Interpolated elevation values on the grid.
    """
    # Flatten grid arrays
    grid_points = np.column_stack((X.ravel(), Y.ravel()))

    # Build KDTree for fast nearest-neighbor lookup
    tree = cKDTree(np.column_stack((x, y)))

    # Query k nearest neighbors for all grid points
    distances, indices = tree.query(grid_points, k=k)

    # Avoid division by zero (replace zeros with a small value)
    distances = np.maximum(distances, 1e-10)

    # Compute IDW weights
    weights = 1.0 / (distances ** power)

    # Normalize weights
    weights /= np.sum(weights, axis=1, keepdims=True)

    # Compute weighted sum of elevations
    Z_interpolated = np.sum(weights * z[indices], axis=1)

    return Z_interpolated.reshape(X.shape)

In [ ]:
from pykrige.ok import OrdinaryKriging
import numpy as np

# ---- Kriging Interpolation ----
def kriging_interpolation(x, y, z, X, Y, sample_size=5000):
    """
    Performs Kriging interpolation with downsampling to handle large datasets.

    Pros: Statistically optimal, handles spatial correlation
    Cons: Computationally expensive, requires variogram tuning

    Parameters
    ----------
    x, y, z : np.ndarray
        Known data points (coordinates and elevation).
    X, Y : np.ndarray
        Grid points where interpolation is applied.
    sample_size : int, optional
        Number of points to sample for Kriging (default is 5000).

    Returns
    -------
    np.ndarray
        Interpolated elevation values on the grid.
    """
    try:
        # Downsample points if there are too many
        if len(x) > sample_size:
            indices = np.random.choice(len(x), sample_size, replace=False)
            x_sample, y_sample, z_sample = x[indices], y[indices], z[indices]
        else:
            x_sample, y_sample, z_sample = x, y, z

        # Apply Kriging only on sampled points
        kriging = OrdinaryKriging(
            x_sample, y_sample, z_sample,
            variogram_model="spherical", verbose=False, enable_plotting=False
        )

        # Interpolate
        Z_pred, _ = kriging.execute("grid", X[0], Y[:, 0])

        return Z_pred

    except MemoryError:
        raise Exception("Kriging interpolation failed: Not enough memory, try a smaller sample size or a different method.")
    except Exception as e:
        raise Exception(f"Kriging interpolation failed: {e}")


In [ ]:
from pykrige.ok import OrdinaryKriging
import numpy as np

# ---- Kriging Interpolation ----
def kriging_interpolation(x, y, z, X, Y, sample_size=5000):
    """
    Performs Kriging interpolation with downsampling to handle large datasets.

    Pros: Statistically optimal, handles spatial correlation
    Cons: Computationally expensive, requires variogram tuning

    Parameters
    ----------
    x, y, z : np.ndarray
        Known data points (coordinates and elevation).
    X, Y : np.ndarray
        Grid points where interpolation is applied.
    sample_size : int, optional
        Number of points to sample for Kriging (default is 5000).

    Returns
    -------
    np.ndarray
        Interpolated elevation values on the grid.
    """
    try:
        # Downsample points if there are too many
        if len(x) > sample_size:
            indices = np.random.choice(len(x), sample_size, replace=False)
            x_sample, y_sample, z_sample = x[indices], y[indices], z[indices]
        else:
            x_sample, y_sample, z_sample = x, y, z

        # Apply Kriging only on sampled points
        kriging = OrdinaryKriging(
            x_sample, y_sample, z_sample,
            variogram_model="spherical", verbose=False, enable_plotting=False
        )

        # Interpolate
        Z_pred, _ = kriging.execute("grid", X[0], Y[:, 0])

        return Z_pred

    except MemoryError:
        raise Exception("Kriging interpolation failed: Not enough memory, try a smaller sample size or a different method.")
    except Exception as e:
        raise Exception(f"Kriging interpolation failed: {e}")


In [ ]:
from scipy.spatial import Delaunay

def tin_interpolation(x, y, z, X, Y):
    """
    Performs TIN (Triangulated Irregular Network) interpolation.

    Pros: Uses Delaunay Triangulation, preserves terrain break lines.
          Ideal for mountainous or hilly terrain where breaklines matter.

    Cons: Less smooth in flat areas, slower than IDW.

    Parameters
    ----------
    x, y, z : np.ndarray
        Known data points (coordinates and elevation).
    X, Y : np.ndarray
        Grid points where interpolation is applied.

    Returns
    -------
    np.ndarray
        Interpolated elevation values on the grid.
    """
    tri = Delaunay(np.column_stack((x, y)))
    return griddata((x, y), z, (X, Y), method="linear", fill_value=np.nan)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

def knn_interpolation(x, y, z, X, Y, k: int = 5):
    """
    Performs K-Nearest Neighbors (KNN) interpolation for DTM generation.

    Pros: Very fast, handles large datasets efficiently.
          Best for categorical terrain classifications or fast previewing.

    Cons: Creates step-like artifacts, not smooth.

    Parameters
    ----------
    x, y, z : np.ndarray
        Known data points (coordinates and elevation).
    X, Y : np.ndarray
        Grid points where interpolation is applied.
    k : int, optional
        Number of nearest neighbors to consider (default is 5).

    Returns
    -------
    np.ndarray
        Interpolated elevation values on the grid.
    """
    # Reshape grid points
    grid_points = np.column_stack((X.ravel(), Y.ravel()))

    # Train KNN model
    knn = KNeighborsRegressor(n_neighbors=k, weights="distance")
    knn.fit(np.column_stack((x, y)), z)

    # Predict elevation values
    Z_pred = knn.predict(grid_points)
    return Z_pred.reshape(X.shape)

In [ ]:
# Plot a DTM
def plot_dtm(Z: np.ndarray, x_range: tuple, y_range: tuple, title: str = "Digital Terrain Model (DTM)") -> None:
    """
    Plots the Digital Terrain Model (DTM) as a heatmap.

    Parameters
    ----------
    Z : np.ndarray
        The DTM grid (2D array) containing elevation values.
    x_range : tuple
        The (min_x, max_x) range for the x-axis.
    y_range : tuple
        The (min_y, max_y) range for the y-axis.
    title : str, optional
        The title of the plot (default is "Digital Terrain Model (DTM)").

    Returns
    -------
    None
        Displays the DTM plot.
    """
    try:
        if Z is None or Z.size == 0:
            raise ValueError("DTM data is empty or not generated.")

        plt.figure(figsize=(10, 7))
        plt.imshow(Z, extent=[x_range[0], x_range[1], y_range[0], y_range[1]],
                   origin="lower", cmap="terrain")
        plt.colorbar(label="Elevation (m)")
        plt.title(title)
        plt.xlabel("X Coordinate")
        plt.ylabel("Y Coordinate")
        plt.show()

    except Exception as e:
        raise Exception(f"Error in plotting DTM: {e}")

In [ ]:
# Plot a DTM
def plot_dtm(Z: np.ndarray, x_range: tuple, y_range: tuple, title: str = "Digital Terrain Model (DTM)") -> None:
    """
    Plots the Digital Terrain Model (DTM) as a heatmap.

    Parameters
    ----------
    Z : np.ndarray
        The DTM grid (2D array) containing elevation values.
    x_range : tuple
        The (min_x, max_x) range for the x-axis.
    y_range : tuple
        The (min_y, max_y) range for the y-axis.
    title : str, optional
        The title of the plot (default is "Digital Terrain Model (DTM)").

    Returns
    -------
    None
        Displays the DTM plot.
    """
    try:
        if Z is None or Z.size == 0:
            raise ValueError("DTM data is empty or not generated.")

        plt.figure(figsize=(10, 7))
        plt.imshow(Z, extent=[x_range[0], x_range[1], y_range[0], y_range[1]],
                   origin="lower", cmap="terrain")
        plt.colorbar(label="Elevation (m)")
        plt.title(title)
        plt.xlabel("X Coordinate")
        plt.ylabel("Y Coordinate")
        plt.show()

    except Exception as e:
        raise Exception(f"Error in plotting DTM: {e}")

In [ ]:
import laspy
import numpy as np

def save_dtm_as_laz(dtm_grid: np.ndarray, transform, output_laz: str):
    """
    Saves a Digital Terrain Model (DTM) as a .laz file.

    Parameters
    ----------
    dtm_grid : np.ndarray
        The DTM elevation grid.
    transform : rasterio.transform.Affine
        Transform containing the geospatial reference information.
    output_laz : str
        Output path for the .laz file.

    Returns
    -------
    None
        Saves a LiDAR file with ground points representing the DTM.
    """
    try:
        # Get grid dimensions
        rows, cols = dtm_grid.shape
        x_min, y_max = transform.c, transform.f  # Top-left corner
        resolution = transform.a  # Grid resolution

        # Generate X, Y coordinates for each grid cell
        x_coords = np.tile(np.arange(cols) * resolution + x_min, rows)
        y_coords = np.repeat(y_max - np.arange(rows) * resolution, cols)

        # Flatten DTM values
        z_coords = dtm_grid.flatten()

        # Remove NaN values (if any)
        valid_mask = ~np.isnan(z_coords)
        x_coords, y_coords, z_coords = x_coords[valid_mask], y_coords[valid_mask], z_coords[valid_mask]

        # Create LAS header
        header = laspy.LasHeader(point_format=1, version="1.2")
        header.scales = (0.01, 0.01, 0.01)  # Set scale factors
        header.offsets = (np.min(x_coords), np.min(y_coords), np.min(z_coords))

        # Create LAS data
        las = laspy.LasData(header)
        las.x = x_coords
        las.y = y_coords
        las.z = z_coords
        las.classification = np.full_like(z_coords, 2, dtype=np.uint8)  # Classify as ground (2)

        # Save to LAZ file
        with laspy.open(output_laz, mode="w", header=header) as las_writer:
            las_writer.write_points(las.points)

        print(f"DTM saved as LAZ: {output_laz}")

    except Exception as e:
        raise Exception(f"Error saving DTM as LAZ: {e}")

In [ ]:
import laspy
import numpy as np

def generate_dtm(input_laz: str, output_resolution: float, method: str = "IDW") -> np.ndarray:
    """
    Generates a Digital Terrain Model (DTM) using ground points (class label = 2) from a .laz file.

    Parameters
    ----------
    input_laz : str
        Path to the input .laz file.
    output_resolution : float
        Grid resolution for the DTM (e.g., 1.0 for 1m resolution).
    method : str, optional
        Interpolation method, either "IDW" or "Kriging". Default is "IDW".

    Returns
    -------
    np.ndarray
        The interpolated DTM grid.

    Raises
    ------
    ValueError
        If an invalid interpolation method is given.
    FileNotFoundError
        If the input file does not exist.
    """
    try:
        if not input_laz.endswith(".laz") and not input_laz.endswith(".las"):
            raise ValueError("Invalid file format. Must be .laz or .las")

        # Read .laz file
        with laspy.open(input_laz) as las_file:
            las = las_file.read()
        print(f"The LAZ file has been loaded.")

        #Extract ground points (classification label 2)
        mask = las.classification == 2
        x, y, z = las.x[mask], las.y[mask], las.z[mask]

        if len(x) == 0:
            raise ValueError("No ground points (class 2) found in the dataset.")
        print(f"Ground points obtained: {len(x)} points")

        # Define grid extent
        min_x, max_x = np.min(x), np.max(x)
        min_y, max_y = np.min(y), np.max(y)

        # Create grid (ensure correct Y-ordering)
        x_grid = np.arange(min_x, max_x + output_resolution, output_resolution)
        y_grid = np.arange(min_y, max_y + output_resolution, output_resolution)[::-1]  # Flip Y-order

        X, Y = np.meshgrid(x_grid, y_grid)
        Z = np.full(X.shape, np.nan)  # Initialize with NaNs

        # Choose interpolation method
        if method.upper() == "IDW":
            Z = idw_interpolation(x, y, z, X, Y)
        elif method.upper() == "KRIGING":
            Z = kriging_interpolation(x, y, z, X, Y)
        elif method.upper() == "NNI":
            Z = natural_neighbor_interpolation(x, y, z, X, Y)
        elif method.upper() == "SPLINE":
            Z = spline_interpolation(x, y, z, X, Y)
        elif method.upper() == "TIN":
            Z = tin_interpolation(x, y, z, X, Y)
        elif method.upper() == "KNN":
            Z = knn_interpolation(x, y, z, X, Y, k=5)  # Using 5 nearest neighbors
        else:
            raise ValueError("Invalid method. Choose from 'IDW', 'Kriging', 'NNI', 'Spline', 'TIN', 'KNN'.")

        print("DTM grid has been generated.")
        return Z

    except FileNotFoundError as e:
        raise FileNotFoundError(str(e))
    except ValueError as e:
        raise ValueError(str(e))
    except Exception as e:
        raise Exception(f"Error generating DTM: {e}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_dtm(Z: np.ndarray, x_range: tuple, y_range: tuple, title: str = "Digital Terrain Model (DTM)") -> None:
    """
    Plots the Digital Terrain Model (DTM) as a heatmap.

    Parameters
    ----------
    Z : np.ndarray
        The DTM grid (2D array) containing elevation values.
    x_range : tuple
        The (min_x, max_x) range for the x-axis.
    y_range : tuple
        The (min_y, max_y) range for the y-axis.
    title : str, optional
        The title of the plot (default is "Digital Terrain Model (DTM)").

    Returns
    -------
    None
        Displays the DTM plot.
    """
    try:
        if Z is None or Z.size == 0:
            raise ValueError("DTM data is empty or not generated.")

        # Flip the DTM grid to match GeoTIFF orientation
        #Z_flipped = np.flipud(Z)

        plt.figure(figsize=(10, 7))
        plt.imshow(Z, extent=[x_range[0], x_range[1], y_range[0], y_range[1]],
                   origin="upper", cmap="terrain")  # Use origin="upper" for correct alignment
        plt.colorbar(label="Elevation (m)")
        plt.title(title)
        plt.xlabel("X Coordinate")
        plt.ylabel("Y Coordinate")
        plt.show()

    except Exception as e:
        raise Exception(f"Error in plotting DTM: {e}")

In [ ]:
import rasterio
from rasterio.transform import from_origin

def save_dtm_as_geotiff(Z: np.ndarray, x_range: tuple, y_range: tuple, resolution: float, output_file: str, crs: str = "EPSG:28992") -> None:
    """
    Saves a Digital Terrain Model (DTM) grid as a GeoTIFF file with a specified CRS.

    Parameters
    ----------
    Z : np.ndarray
        The DTM grid (2D array) containing elevation values.
    x_range : tuple
        The (min_x, max_x) range for the x-axis.
    y_range : tuple
        The (min_y, max_y) range for the y-axis.
    resolution : float
        Grid resolution (e.g., 1.0 for 1m resolution).
    output_file : str
        Path to save the GeoTIFF file.
    crs : str, optional
        Coordinate Reference System (CRS) in EPSG format (default is "EPSG:28992").

    Returns
    -------
    None
        Saves the DTM as a GeoTIFF file.
    """
    try:
        # Define transform (top-left corner as origin)
        transform = from_origin(x_range[0], y_range[1], resolution, resolution)

        # Save as GeoTIFF using rasterio
        with rasterio.open(
            output_file,
            "w",
            driver="GTiff",
            height=Z.shape[0],
            width=Z.shape[1],
            count=1,
            dtype=Z.dtype,
            crs=crs,  # Use provided CRS
            transform=transform
        ) as dst:
            dst.write(Z, 1)  # Write the DTM to band 1

        print(f"DTM saved as GeoTIFF: {output_file} with CRS {crs}")

    except Exception as e:
        raise Exception(f"Error saving GeoTIFF: {e}")


In [ ]:
# Mount and connect to the Google Drive
from google.colab import drive
drive.mount('/content/drive')
# "/content/drive/MyDrive/..."

In [ ]:
# Example of Usage
#input_laz = "../../Datasets/24HN1_25.laz"  # Replace with your LiDAR file path
input_laz = "/content/drive/MyDrive/Colab_Notebooks/AWD/datasets/24HN1_25/1_original/24HN1_25.laz"

# Set the DTM resolution in meters
resolution = 10.0

# Generate and visualize DTM using KNN
# Here, six options are available, i.e., IDW, NNI, KRIGING, SPLINE, TIN, KNN
dtm = generate_dtm(input_laz, resolution, method="IDW")

In [ ]:
# To visualize the generated DTM and output it as a geotiff file.

# Get the min/max values for plotting
with laspy.open(input_laz) as las_file:
    las = las_file.read()
    mask = las.classification == 2
    min_x, max_x = np.min(las.x[mask]), np.max(las.x[mask])
    min_y, max_y = np.min(las.y[mask]), np.max(las.y[mask])

# Plot the DTM
plot_dtm(dtm, (min_x, max_x), (min_y, max_y), title="DTM")

# Output GeoTIFF filename
output_tif = "/content/drive/MyDrive/Colab_Notebooks/AWD/datasets/24HN1_25/1_original/24HN1_25.tif"

# Example: The Dutch Amersfoort/RD New CRS.
desired_crs = "EPSG:28992"

# Save the DTM as a GeoTIFF file
save_dtm_as_geotiff(dtm, (min_x, max_x), (min_y, max_y), resolution, output_tif,crs=desired_crs)